In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/

/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI


In [ ]:
# from HelperFunctions import *
# readNapkinStrokesToCSV('/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/NewTrainSet/data.json', '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/NewTrainSet')


In [2]:
from GraphicManager import GraphicManager
from GestuReNN_mts import GestuReNN_GRU #GestuReNN_mts_without_regression as GestuReNN
from DataLoader import DataLoader
import numpy as np
import time
import json

import tensorflow as tf
print('no. of gpus  ', tf.config.experimental.list_physical_devices("GPU"))


# test on old recognition dataset
class Parameters:
    pad = True
    include_fingerup = False
    model_inputs = 'coord_and_tang'
    test_size = 0.2
    dataset = 'Napkin'
    load_mode = 'test'
    augmentFactor = 0
    datasetFolder = '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/TestCSV'
    fileType = 'csv'
    labelJsonPath = '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/labelDict_2_classes.json'
    batchSize = 64
    excludeClasses = ['line']
    modelPath = '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/checkpoints/models/binary_classification_arrow'

# test on validation set of training data
# class Parameters:
#     pad = True
#     include_fingerup = True
#     use_tangents = True
#     test_size = 0.2
#     method = 'G3'
#     dataset = 'Napkin'
#     load_mode = 'validation'
#     augmentFactor = 0
#     datasetFolder = '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/TrainCSV'
#     fileType = 'csv'
#     labelJsonPath = '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/test/labelDict_10_classes.json'
#     batchSize = 128
#     excludeClasses = ['line']
#     modelPath = '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/checkpoints/models/Napkin_trial'


# train on Napkin dataset.
# class Parameters:
#     pad = True
#     include_fingerup = False
#     model_inputs = 'coord_and_tang'
#     test_size = 0.2
#     dataset = 'Napkin'
#     load_mode = 'train'
#     augmentFactor = 3
#     datasetFolder = '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/NewTrainSet/csv'
#     fileType = 'csv'
#     labelJsonPath = '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/labelDict_2_classes.json'
#     batchSize = 32
#     excludeClasses = ['line']
#     modelPath = '/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/checkpoints/models/binary_classification_arrow'


p = Parameters()

no. of gpus   []


In [3]:
# Data loading
dl = DataLoader(dataset=p.dataset, load_mode=p.load_mode, labelJsonPath=None, datasetFolder=p.datasetFolder,
                fileType=p.fileType, include_fingerup=p.include_fingerup, model_inputs=p.model_inputs,
                augmentFactor=p.augmentFactor, excludeClasses=p.excludeClasses)

print(dl.validation_set[0].shape)
print(dl.train_set[0].shape)
for k, v in dl.labels_dict.items():
    print('{} - {}'.format(v, k))

Starting the DataLoader construction ...
stroke_dataset - Napkin
.. Done with attribute settings. Loading the data ...
42.55%- Loading on /content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/TestCSV
85.11%- Loading on /content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/TestCSV
Loading on /content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/dataset/NapkinData/TestCSV completed.
{'non-arrow': 0, 'arrow': 1}
.. Done with data loading. Setting up classifier attributes ...
Done with DataLoader construction!
(234, 64, 4)
(234, 64, 4)
0 - non-arrow
1 - arrow


/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/DataLoader.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x, y = np.array(x), np.array(y)
/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/DataLoader.py:216: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)


In [4]:
import random
x_train, y_train = dl.train_set
resampled_x_train, resampled_y_train = [],[]
for i in range(x_train.shape[0]):
  if y_train[i]==1:
    if random.uniform(0, 1) < 0.3:
      resampled_x_train.append(x_train[i])
      resampled_y_train.append(y_train[i])
  else:
    resampled_x_train.append(x_train[i])
    resampled_y_train.append(y_train[i])
train_set = np.array(resampled_x_train), np.array(resampled_y_train)


In [15]:
# with open(p.labelJsonPath, 'w') as fp:
#     json.dump(dl.labels_dict, fp)

In [5]:
#print classwise training samples
temp = train_set[1]
print(temp.shape)
countDict = {}
for i in range(temp.shape[0]):
  try:
    countDict[temp[i]] +=1
  except:
    countDict[temp[i]] =1
x = [i for i in range(2)]
y = [countDict[i] for i in x]
for k in x:
  print("{}: {}".format(k, countDict[k]))


(3383,)
0: 952
1: 2431


In [4]:
# model initialization.
model = GestuReNN_GRU(plot=False, labelJsonPath=p.labelJsonPath, model_inputs=p.model_inputs,
                      batch_size=p.batchSize, model_path=p.modelPath, include_fingerup=p.include_fingerup)
graphic_manager = GraphicManager(dataset=p.dataset)

----#classes = 2------


In [5]:
if p.load_mode == 'train':
    with open(p.labelJsonPath, 'w') as fp:
        json.dump(dl.labels_dict, fp)
    model.fit_model(train_set, dl.validation_set)
else:
    print("loading model ...")
    model.load_model()
    print('Model Loaded!!')
    graphic_manager.generate_progressive_accuracy(model, dl.test_set)

loading model ...
/content/drive/MyDrive/second_layer_ai_model/SketchAI/SketchAI/checkpoints/models/binary_classification_arrow/mdcp_robust.ckpt
Model Loaded!!
class: non-arrow, accuracy: 0.018691588785046728, total: 214
class: arrow, accuracy: 0.4, total: 20
